In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score , mean_squared_error

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

import pickle

In [3]:
# Lets read the proeccessed data 

df=pd.read_csv("C:/Users/Daya/Desktop/DataSets/AQI/final_data.csv")

In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,0.022013,0.000000,0.146789,0.823344,0.935897,0.036364,0.162500,0.134650,219.720833
1,0.034591,0.081232,0.134557,0.851735,0.858974,0.054545,0.166667,0.165171,182.187500
2,0.000000,0.100840,0.073394,0.880126,0.794872,0.054545,0.183333,0.165171,154.037500
3,0.059748,0.159664,0.100917,0.858044,0.666667,0.090909,0.320833,0.335727,223.208333
4,0.179245,0.310924,0.134557,0.813880,0.525641,0.181818,0.345833,0.364452,200.645833


In [5]:
# Independent Features 
x=df.drop('PM 2.5',axis=1)

# Dependent Features 
y=df['PM 2.5']

In [6]:
# SPlit the data in train and test

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

# Decision Tree Regressor

In [9]:
from sklearn.tree import DecisionTreeRegressor

tree_regression=DecisionTreeRegressor()

# We will try wihtout gird search cv 

In [10]:
tree_regression.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [11]:
tree_pred=tree_regression.predict(x_test)

In [12]:
print("ACCURACY:", r2_score(y_test, tree_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, tree_pred)))
print("MSE:", mean_squared_error(y_test, tree_pred))

ACCURACY: 0.6339575343588387
RMSE: 47.880549759600534
MSE: 2292.547045281583


In [13]:
# Let's try out grid search and see whether we can get better result than this

In [14]:
dtr_params = dict(max_depth=[3,5,8,10,15,17,None], max_features=["sqrt","log2","auto",None],
                  max_leaf_nodes=[None, 10,20,30,40,50,60,70,80,90,100],
                  min_weight_fraction_leaf = [0.0,0.1,0.5,0.2,0.25,0.3,0.35,0.4,0.5],
                  splitter=['best', "random"],min_samples_leaf=[1,2,3,4,5] )

In [15]:
tree_regressor=GridSearchCV(tree_regression,cv=5,param_grid=dtr_params,verbose=1,scoring="neg_mean_squared_error")

In [16]:
tree_regressor.fit(x_train,y_train)

tree_pred=tree_regressor.predict(x_test)

Fitting 5 folds for each of 27720 candidates, totalling 138600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 138600 out of 138600 | elapsed:  7.1min finished


In [17]:
tree_regressor.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_weight_fraction_leaf': 0.0,
 'splitter': 'best'}

In [18]:
print("ACCURACY:", r2_score(y_test, tree_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, tree_pred)))
print("MSE:", mean_squared_error(y_test, tree_pred))

ACCURACY: 0.517301251102654
RMSE: 54.98338570665188
MSE: 3023.17270376645


# Random Forest Regression

In [19]:
from sklearn.ensemble import RandomForestRegressor

ran_for= RandomForestRegressor()

ran_for.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [20]:
ren_pred= ran_for.predictict(x_test)

AttributeError: 'RandomForestRegressor' object has no attribute 'predictict'

In [ ]:
print("ACCURACY:", r2_score(y_test, ren_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, ren_pred)))
print("MSE:", mean_squared_error(y_test, ren_pred))

### HyperParameter Tuning

In [ ]:

paramters = dict(max_depth=[int(x) for x in np.linspace(5, 30, num = 6)],
                 max_features=['auto', "log2", "sqrt", None],
                 min_samples_leaf=[1,2,5,10,15],
                 min_samples_split=[2, 5, 10,15,100],
                 min_weight_fraction_leaf=[0.0,0.05,0.5],
                 n_estimators=[int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rfr=RandomForestRegressor()
random_forest_regressor = RandomizedSearchCV(rfr, paramters, scoring = "neg_mean_squared_error", cv = 5, verbose = 1,n_iter=100)
random_forest_regressor.fit(x_train,y_train)

In [ ]:
ran_predict=random_forest_regressor.predict(x_test)

In [ ]:
print("ACCURACY:", r2_score(y_test, ran_predict))
print("RMSE:", np.sqrt(mean_squared_error(y_test, ran_predict)))
print("MSE:", mean_squared_error(y_test, ran_predict))

In [ ]:
residual_random_forest = y_test - ran_predict
sns.distplot(residual_random_forest)

In [ ]:
import pickle
random_forest_file = open("random_forest.pkl", "wb")
pickle.dump(random_forest_regressor, random_forest_file)